# Projet Détection d'un objet à l'aide de YOLOV8

## CUDA

In [ ]:
from numba import jit, cuda
import numpy as np
# to measure exec time
from timeit import default_timer as timer   
 
# normal function to run on cpu
def func(a):                                
    for i in range(10000000):
        a[i]+= 1     
 
# function optimized to run on gpu 
@jit(target_backend='cuda')                         
def func2(a):
    for i in range(10000000):
        a[i]+= 1
if __name__=="__main__":
    n = 10000000                           
    a = np.ones(n, dtype = np.float64)
     
    start = timer()
    func(a)
    print("without GPU:", timer()-start)    
     
    start = timer()
    func2(a)
    print("with GPU:", timer()-start)

## Chargement des données

In [ ]:
import os 

DOSSIER_IMAGES = 'DATA/IMAGES/'
DOSSIER_LABELS = 'DATA/LABELS/'

# Récupération des fichiers
images_file = os.listdir(DOSSIER_IMAGES)
nb_images   = len(images_file)

In [ ]:
import random 
import shutil

# Split Train/Test/Valid
PART_TRAIN = 0.7
PART_VALID = 0.1
PART_TEST  = 0.2

DOSSIER_TRAIN = 'datasets/TRAIN/'
DOSSIER_VALID = 'datasets/VALID/'
DOSSIER_TEST  = 'datasets/TEST/'

# Mélange du tableau
random.seed(69)
for _ in range(10): random.shuffle(images_file)

# Création des dossiers Train Test et Valid
if os.path.exists('datasets/'): shutil.rmtree('datasets/')

os.makedirs(DOSSIER_TRAIN)
os.makedirs(DOSSIER_VALID)
os.makedirs(DOSSIER_TEST)

os.makedirs(DOSSIER_TRAIN + 'images/')
os.makedirs(DOSSIER_VALID + 'images/')
os.makedirs(DOSSIER_TEST  + 'images/')

os.makedirs(DOSSIER_TRAIN + 'labels/')
os.makedirs(DOSSIER_VALID + 'labels/')
os.makedirs(DOSSIER_TEST  + 'labels/')

# Remplissage des dossier
train = images_file[:int(nb_images*PART_TRAIN)]
valid = images_file[int(nb_images*PART_TRAIN):int(nb_images*PART_TRAIN) + int(nb_images*PART_VALID)]
test  = images_file[int(nb_images*PART_TRAIN)+int(nb_images*PART_VALID):]

for file in train:
    src_img  = DOSSIER_IMAGES + file
    dest_img = DOSSIER_TRAIN  + 'images/' + file
    shutil.copy(src_img, dest_img)

    file_name  = file.split('.')[0] + '.txt'
    src_label  = DOSSIER_LABELS + file_name
    dest_label = DOSSIER_TRAIN  + 'labels/' + file_name
    shutil.copy(src_label, dest_label)

for file in test:
    src_img  = DOSSIER_IMAGES + file
    dest_img = DOSSIER_TEST  + 'images/' + file
    shutil.copy(src_img, dest_img)

    file_name  = file.split('.')[0] + '.txt'
    src_label  = DOSSIER_LABELS + file_name
    dest_label = DOSSIER_TEST  + 'labels/' + file_name
    shutil.copy(src_label, dest_label)

for file in valid:
    src_img  = DOSSIER_IMAGES + file
    dest_img = DOSSIER_VALID  + 'images/' + file
    shutil.copy(src_img, dest_img)

    file_name  = file.split('.')[0] + '.txt'
    src_label  = DOSSIER_LABELS + file_name
    dest_label = DOSSIER_VALID  + 'labels/' + file_name
    shutil.copy(src_label, dest_label)

## Chargement de YoloV8


### Fichier Yaml

In [ ]:
yaml_content = f'''
train: ./TRAIN/images
val:   ./VALID/images
test:  ./TEST/images

names: ['shoes']
'''

with open('conf_yaml.yaml', 'w') as file:
    file.write(yaml_content)
    file.close()

### Entraînement

In [ ]:
import torch
import os
from ultralytics import YOLO
from numba import jit, cuda

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
    
model   = YOLO("yolov8n-seg.pt")
model.to('cuda')
results = model.train(
        batch=8,
        data="conf_yaml.yaml",
        epochs=1,
        imgsz=640,
    )

torch.save(model, 'shoes-seg.pt')


In [36]:
from ultralytics import YOLO
import torch
import os 
import json
from PIL import Image

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

model = YOLO("runs/segment/train/weights/best.pt")
#img = Image.open('datasets/TEST/images/202.jpg')
#img = img.resize((640, 640))
#img.save('test.jpg')
pred = model.predict('datasets/TEST/images/146.jpg', conf=0.25)



image 1/1 c:\Users\Aurel\Documents\ISEN\M2\Vision_Artificielle_IA\artificial-vision\datasets\TEST\images\146.jpg: 640x640 47 shoess, 168.3ms
Speed: 3.0ms preprocess, 168.3ms inference, 73.7ms postprocess per image at shape (1, 3, 640, 640)


In [35]:
import torchvision.transforms as T
import numpy as np
from PIL import Image

orig_img   = Image.fromarray(pred[0].orig_img)
black_img  = Image.fromarray(np.zeros_like(pred[0].orig_img))
mask_img   = T.ToPILImage()(pred[0].masks.data)

result_img = Image.composite(orig_img, black_img, mask=mask_img)

orig_img.show()
result_img.show()
#pred[0].masks
#T.ToPILImage()(pred[0].masks.data).show()

ValueError: pic should not have > 4 channels. Got 47 channels.

In [33]:
mask      = (pred[0].masks.data).numpy()
(p, h, w) = mask.shape

img_final = pred[0].orig_img

for x in range(h):
    for y in range(w):
        for z in range(p):
            if mask[z][x][y] == 0:
                img_final[x][y][z] = 0

img_final = Image.fromarray(img_final)
img_final.show()

IndexError: index 3 is out of bounds for axis 0 with size 3

In [2]:
import os
import cv2 
import json
import torch
import numpy as np

from PIL import Image
from ultralytics import YOLO

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

#img = Image.open('IMG_20240113_191245.jpg')
#img = img.resize((640, 640))
#img.save('test.jpg')

model = YOLO("runs/segment/train/weights/best.pt")
pred  = model.predict('datasets/TEST/images/194.jpg', conf=0.25)
img   = cv2.imread('datasets/TEST/images/194.jpg')
#img_noir = np.zeros_like(img)

for result in pred:
    for mask in result.masks.xy:
        points = np.int32([mask])
        #cv2.polylines(img, points, True, (255, 0, 0), 1)
        cv2.fillPoly(img, points, [255, 0, 0])

cv2.imshow("Image", img)
cv2.waitKey(0)



image 1/1 c:\Users\Aurel\Documents\ISEN\M2\Vision_Artificielle_IA\artificial-vision\datasets\TEST\images\194.jpg: 640x640 47 shoess, 618.2ms
Speed: 56.1ms preprocess, 618.2ms inference, 228.8ms postprocess per image at shape (1, 3, 640, 640)


-1

In [22]:
img = cv2.imread('datasets/TEST/images/194.jpg')
results = []
for result in pred:
    for mask in result.masks.xy:
        img_class_isolate = np.copy(img)
        points = np.int32([mask])
        cv2.fillPoly(img_class_isolate, points, [255, 0, 0])
        results.append(img_class_isolate)
for i in range(len(results)):
    cv2.imwrite('test' + str(i) + '.jpg',  results[i])

: 

In [42]:
i = 1
for result in pred:
    for mask in result.masks.data:
        # scale for visualizing results
        people_mask = torch.any(mask, dim=0).int() * 255
        # save to file
        cv2.imwrite('ImageTest/' + str(i) + '.jpg', people_mask.cpu().numpy())
        i+=1

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ...

In [84]:
img = cv2.imread('datasets/TEST/images/194.jpg')
for result in pred:
    for i, mask in enumerate(result.masks.data):
        img_copy = np.copy(img)
        img_result = img_copy[torch.any(mask, dim=0).int()]
        cv2.imwrite('ImageTest/' + str(i+1) + '.jpg', img_class_isolate)
        print(str(i+1) + '.jpeg créé')

print(mask_i)

1.jpeg créé
2.jpeg créé
3.jpeg créé
4.jpeg créé
5.jpeg créé
6.jpeg créé
7.jpeg créé
8.jpeg créé
9.jpeg créé
10.jpeg créé
11.jpeg créé
12.jpeg créé
13.jpeg créé
14.jpeg créé
15.jpeg créé
16.jpeg créé
17.jpeg créé
18.jpeg créé
19.jpeg créé
20.jpeg créé
21.jpeg créé
22.jpeg créé
23.jpeg créé
24.jpeg créé
25.jpeg créé
26.jpeg créé
27.jpeg créé
28.jpeg créé
29.jpeg créé
30.jpeg créé
31.jpeg créé
32.jpeg créé
33.jpeg créé
34.jpeg créé
35.jpeg créé
36.jpeg créé
37.jpeg créé
38.jpeg créé
39.jpeg créé
40.jpeg créé
41.jpeg créé
42.jpeg créé
43.jpeg créé
44.jpeg créé
45.jpeg créé
46.jpeg créé
47.jpeg créé
tensor([ True,  True,  True,  True,  True, False])


In [96]:
torch.where(pred[0].boxes.data[:, 5] != -1)
len(torch.any(pred[0].masks.data[0], dim=[0,1]).int())

(tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46]),)

In [36]:
for result in pred:

    masks = result.masks.data
    boxes = result.boxes.data

    clss = boxes[:, 5]

    obj_indices = torch.where(clss == 0)
    obj_masks = masks[obj_indices]
    obj_mask = torch.any(obj_masks, dim=0).int()
    res = img*obj_mask.numpy()[:, :, np.newaxis]
    alpha = np.sum(res, axis=-1) > 0
    alpha = np.uint8(alpha * 255)
    res = np.dstack((res, alpha))

    cv2.imwrite(str(f'ImageTest/all_shoes.png'), res)

    #MASK FOR EACH INSTANCE OF A CLASS
    for i, obj_index in enumerate(obj_indices[0].numpy()):
        obj_masks = masks[torch.tensor([obj_index])]
        obj_mask = torch.any(obj_masks, dim=0).int()
        res = img*obj_mask.numpy()[:, :, np.newaxis].astype(np.uint8)
        # Créer un masque pour les pixels noirs
        black_pixels_mask = (res == [0, 0, 0])
        # Remplacer les pixels noirs par la couleur (255, 255, 255, 0)
        res[black_pixels_mask] = [255, 255, 255, 0]
        cv2.imwrite(str(f'ImageTest/{i}.jpg'), res.astype(np.uint8))

ValueError: NumPy boolean array indexing assignment cannot assign 4 input values to the 1221147 output values where the mask is true

In [20]:
img   = cv2.imread('datasets/TEST/images/194.jpg')

box  = pred[0].boxes[0].xyxy[0]
min_x = box[0].int()
min_y = box[1].int()
max_x = box[2].int()
max_y = box[3].int()

cropped_img = img[min_y:max_y,min_x:max_x]
cv2.imwrite('test.png', cropped_img)

True